In [ ]:
!pip install rpy2==3.5.1 #install package that linkage python with R

!apt-get install r-base

%load_ext rpy2.ipython

# Import rpy2 for dataframe conversion
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import globalenv
import ee
import pandas as pd
from IPython.display import Image
import ee, datetime
from pylab import *
from matplotlib.pylab import rcParams
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
from math import pi, atan2, cos, sin, sqrt
import scipy as sp
from scipy.interpolate import interp1d
import os
import glob
from scipy import stats
from google.colab import files

#login through your personal gee account
#After running this cell, click on the link below, generate a token and than insert the square below and press enter
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp39-cp39-linux_x86_64.whl size=317919 sha256=7f93d96211d1720932cd06ed0c6c9776652dbb85470836c3a3434150c613fd0c
  Stored in directory: /root/.cache/pip/wheels/09/e7/bc/33685b60ab54dba969596dd87244ee9f4c2e83dff9a53d4f20
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5
Reading package lists... Done
Building dependency tree       
Reading state information... Done
r-base is already the newest version (4.2.2.20221110-1.2004.0).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web

In [ ]:
# #option 1: enter your coordinates:
# #option 2: use the map below to choose your coordinats

# #option 1:
# long = 17.603754704680803
# lat = -27.771703545760328

# #enter the river name
# river_name = "Fish"

# #choose month of hydrological year:
# hydrological_year = 1

# # Convert the latitude and longitude to a Point geometry
# point = {'type':'Point', 'coordinates':[ long, lat]};

In [ ]:
#
# #installing the package of GEE
!pip install geemap 

#upload GEE map of NDVI from MODIS, add layers of NDVI & STD
#choose your pixel from hyper arid river with thigh values of std (greener)

import geemap
map1 = geemap.Map()
map1.add_basemap('SATELLITE')  #change the base map to defult satellite of google

#adding hyperarid regions mask (<100mm/year)
dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filter(ee.Filter.date('2000-01-01', '2021-01-01'));
                 
precipitation = dataset.select('precipitation');
all = precipitation.reduce(ee.Reducer.sum());
average_all = all.divide(21);
vis = {
  'min': 0.0,
  'max': 200.0,
  'palette': ['faff00', '3bb63d'],
  'opacity': 0.5
};

map1.addLayer(average_all, vis, 'Precipitation');

#adding STD of NDVI map
ndvi = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate('2001-01-01', '2022-01-01').select('NDVI'); #use NDVI from MODIS

def multiply_by_0_0001(img):
    return img.multiply(0.0001)

ndvi = ndvi.map(multiply_by_0_0001) #scaling the imagecollection to values between: -1 to 1


std_ndvi = ndvi.reduce(ee.Reducer.stdDev()) #doing std for each pixel

palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301'];

vis_params_ndvi = {'min': 0, 'max': 1, 'palette': palette} #parameters for vizualize NDVI
#map1.addLayer(ndvi, vis_params_ndvi, 'NDVI')            # unmark to add NDVI layer

vis_params_std = {'min': 0, 'max': 0.08, 'opacity': 0.5, 'palette': palette} #parameters for vizualize STD
map1.addLayer(std_ndvi, vis_params_std , 'STD');             # add layer

map1.add_colorbar(vis_params_std, label="STD", layer_name="colorbar_std") #adding colorbar of STD
map1.addLayerControl()   #adding control of layer on the upper right side of map

map1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
#option 2:
# Draw a marker on the map above, on your choosen pixel, and than activate this cell:
roi = map1.draw_last_feature

latlon = roi.getInfo()

long = latlon['geometry']["coordinates"][0]
lat = latlon['geometry']["coordinates"][1]

# Convert the latitude and longitude to a Point geometry
point = {'type':'Point', 'coordinates':[ long, lat]};

#enter the river name
river_name = "rooibank"

#choose month of hydrological year:
hydrological_year = 1

In [ ]:
#choose the dates
start_date = '2001-01-01'
end_date = '2022-12-31'

#choose the product with relevent bands for MSAVI
image = ee.ImageCollection("MODIS/006/MOD09GQ").filterDate(start_date, end_date).select('sur_refl_b01','sur_refl_b02')

#calculate the index
def msavi_calculate(img):
    return img.expression(
                      '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2', {
                        'NIR' : img.select('sur_refl_b02'),
                        'RED' : img.select('sur_refl_b01')
                      }).rename('MSAVI')

msavi = image.map(msavi_calculate)

msavi = msavi.select('MSAVI')

#extract the index for the choosen coordinate point
info = msavi.getRegion(point,250).getInfo() # 250m resolution

# Convert the 'info' object into a pandas DataFrame
df = pd.DataFrame(info[1:], columns=info[0])

# Extract the 'date' and 'MSAVI' columns
df = df[['id', 'MSAVI']]
df = df.rename(columns={'id':'date'})

# Convert the 'date' column into a pandas datetime object
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'], format='%Y_%m_%d')

# Set the 'date' column as the index of the DataFrame
#df.set_index('date', inplace=True)

#give unique name for df
msavi_modis = df

#choose the dates
start_date = '2001-01-01'
end_date = '2022-12-31'

#choose the product with NDWI (same dates)
ndwi = ee.ImageCollection("MODIS/MCD43A4_006_NDWI").filterDate(start_date, end_date).select('NDWI');

#extract the index for the choosen coordinate point
info = ndwi.getRegion(point,464).getInfo() # 464m resolution

# Convert the 'info' object into a pandas DataFrame
df = pd.DataFrame(info[1:], columns=info[0])

# Extract the 'date' and 'NDWI' columns
df = df[['id', 'NDWI']]
df = df.rename(columns={'id':'date'})

# Convert the 'date' column into a pandas datetime object
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'], format='%Y_%m_%d')

# Set the 'date' column as the index of the DataFrame
#df.set_index('date', inplace=True)

#give unique name for df
ndwi_modis = df

#choose the dates
start_date = '1984-04-01'
end_date = '2000-12-31'

cloud_perc = 10

image = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filterDate(start_date, end_date).filter(ee.Filter.lt('CLOUD_COVER', cloud_perc));

#calculate the index
def msavi_calculate(img):
    return img.expression(
                      '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2', {
                        'NIR' : img.select('SR_B4'),
                        'RED' : img.select('SR_B3')
                      }).rename('MSAVI')

msavi = image.map(msavi_calculate)

msavi = msavi.select('MSAVI')

#extract the index for the choosen coordinate point
info = msavi.getRegion(point,30).getInfo() # 30m resolution

# Convert the 'info' object into a pandas DataFrame
df = pd.DataFrame(info[1:], columns=info[0])

# Extract the 'date' and 'MSAVI' columns
df = df[['id', 'MSAVI']]
df = df.rename(columns={'id':'date'})

# Convert the 'date' column into a pandas datetime object
df['date'] = df['date'].astype(str)
def extract_date(string):
    year = string[12:16]
    month = string[16:18]
    day = string[18:20]
    return year + "_" + month + "_" + day

df['date'] = df['date'].apply(extract_date)
df['date'] = pd.to_datetime(df['date'], format='%Y_%m_%d')

# Set the 'date' column as the index of the DataFrame
#df.set_index('date', inplace=True)

#give unique name for df
msavi_landsat = df.sort_values(by='date').reset_index(drop=True)

#choose the dates
start_date = '1982-01-01'
end_date = '2000-12-31'

ndvi = ee.ImageCollection("NOAA/CDR/AVHRR/NDVI/V5").filterDate(start_date, end_date).select('NDVI');

def multiply_by_0_0001(img):
    return img.multiply(0.0001)

ndvi = ndvi.map(multiply_by_0_0001) #scaling the imagecollection to values between: -1 to 1

#extract the index for the choosen coordinate point
info = ndvi.getRegion(point,5566).getInfo() # 5566m resolution

# Convert the 'info' object into a pandas DataFrame
df = pd.DataFrame(info[1:], columns=info[0])

# Extract the 'date' and 'NDWI' columns
df = df[['id', 'NDVI']]
df = df.rename(columns={'id':'date'})

# Convert the 'date' column into a pandas datetime object
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Set the 'date' column as the index of the DataFrame
df.set_index('date', inplace=False)

#give unique name for df
ndvi_avhrr = df

#choosing the df of AVHRR NDVI and MODIS NDWI
df = ndvi_avhrr  
df2 = ndwi_modis
df3 = msavi_modis

# Convert the python dataframe to the R dataframe
with localconverter(ro.default_converter + pandas2ri.converter):
  dfr = ro.conversion.py2rpy(df)

with localconverter(ro.default_converter + pandas2ri.converter):
  dfr2 = ro.conversion.py2rpy(df2)

with localconverter(ro.default_converter + pandas2ri.converter):
  dfr3 = ro.conversion.py2rpy(df3)

# Create a variable name in R's global environment
globalenv['dfr'] = dfr


# Create a variable name in R's global environment
globalenv['dfr2'] = dfr2


# Create a variable name in R's global environment
globalenv['dfr3'] = dfr3



In [ ]:
%%R
# Install R package bfast
install.packages('bfast')
# Import the R package
library(bfast)

df = dfr2

df[is.na(df)] = 0
sp_vector <- as.vector(df['NDWI'])
vec1 <- df$NDWI
sp_ts <- ts(vec1,start=c(2001,1),frequency=365.1)

fit <- bfast(sp_ts, h=0.1 , season = "harmonic", max.iter = 1)
#plot(fit)
library(tidyverse)
fit[["output"]][[1]][["Tt"]]
fit[["output"]][[1]][["St"]]
fit[["output"]][[1]][["Nt"]]
fit[["output"]][[1]][["Vt"]]
fit[["output"]][[1]][["bp.Vt"]][["breakpoints"]]


dat = fit[["output"]][[1]][["Tt"]]
Tt = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["St"]]
St = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["Nt"]]
Nt = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["Vt"]]
Vt = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["bp.Vt"]][["breakpoints"]]
Bt = data.frame(Y=as.matrix(dat), date=time(dat))

Tt_l =Tt["Y"]#Tt
St_l = St["Y"]#St
Nt_l = Nt["Y"]#et
Vt_l = Vt["Y"]#Vt
Bt_l = Bt["Y"]#Bt
Dt_l= df['date']

length(Dt_l) <- length(St_l)


df_csv<-data.frame(Dt_l,Tt_l,St_l,Nt_l,Vt_l)

mags <- fit[["Mags"]]
df_csv2<-data.frame(mags,Bt_l)

df = dfr

df[is.na(df)] = 0
sp_vector <- as.vector(df['NDVI'])
vec1 <- df$NDVI
sp_ts <- ts(vec1,start=c(1982,1),frequency=365.1)

fit <- bfast(sp_ts, h=0.06 , season = "harmonic", max.iter = 1)
#plot(fit)
library(tidyverse)
fit[["output"]][[1]][["Tt"]]
fit[["output"]][[1]][["St"]]
fit[["output"]][[1]][["Nt"]]
fit[["output"]][[1]][["Vt"]]
fit[["output"]][[1]][["bp.Vt"]][["breakpoints"]]


dat = fit[["output"]][[1]][["Tt"]]
Tt = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["St"]]
St = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["Nt"]]
Nt = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["Vt"]]
Vt = data.frame(Y=as.matrix(dat), date=time(dat))
dat = fit[["output"]][[1]][["bp.Vt"]][["breakpoints"]]
Bt = data.frame(Y=as.matrix(dat), date=time(dat))

Tt_l =Tt["Y"]#Tt
St_l = St["Y"]#St
Nt_l = Nt["Y"]#et
Vt_l = Vt["Y"]#Vt
Bt_l = Bt["Y"]#Bt
Dt_l= df['date']

length(Dt_l) <- length(St_l)


df_csv3<-data.frame(Dt_l,Tt_l,St_l,Nt_l,Vt_l)

#print(df_csv)




mags <- fit[["Mags"]]
df_csv4<-data.frame(mags,Bt_l)

(as ‘lib’ is unspecified)






























































































	‘/tmp/RtmptFey9y/downloaded_packages’



Attaching package: ‘zoo’



    as.Date, as.Date.numeric



  method            from
  as.zoo.data.frame zoo 


✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

✖ stringr::boundary() masks strucchangeRcpp::boundary()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()



In [ ]:
#convert the two dfs to python format of bfast results and breaks + magnitudes results
df_bfast_modis_ndwi = %Rget df_csv  
df_mags_modis_ndwi = %Rget df_csv2 

df_bfast_avhrr_ndvi = %Rget df_csv3  
df_mags_avhrr_ndvi = %Rget df_csv4

def magnitude(bfast, mag):
    
    

    day = list(mag['Y'])
    df_mag = pd.DataFrame(mag)
    magnitude = {}
    c=0
    j=0
    
    #without negative values:
    for  i in bfast['Y']:
        if c in day:
            #mag_points = abs(df_mag['X3'][j])
            mag_points = df_mag['X3'][j]
            if mag_points >0:
                date =bfast['date'][c]
                magnitude[date] = mag_points
                c+=1
                j+=1
            else:
               c+=1 
               j+=1
        else:
                c+=1

    df_mag = pd.DataFrame(magnitude.items(), columns=['date', 'Magnitude']) 
    df_mag['date'] = pd.to_datetime(df_mag['date'])
    df_mag['year'] =  pd.DatetimeIndex(df_mag['date']).year
    df_mag['month'] =  pd.DatetimeIndex(df_mag['date']).month
    j=0
    for i in df_mag['month']:
        if i == 11 or i == 12:
            change = int( df_mag['year'][j])+1
            df_mag['year'][j]= change
            j+=1
        else:
            j+=1
            
  
    return df_mag

df_mag_modis_ndwi = magnitude(df_bfast_modis_ndwi, df_mags_modis_ndwi)
# df_mag_modis_msavi = magnitude(df_bfast_modis_msavi, df_mags_modis_msavi)
df_mag_avhrr_ndvi = magnitude(df_bfast_avhrr_ndvi, df_mags_avhrr_ndvi)

#equations of magnitude:
volume_modis_ndwi = lambda x: -8.439 + 1139.426 * x
duration_modis_ndwi = lambda x: -0.123 + 295.356 * x

# volume_modis_msavi = lambda x: -22.993 + 718.703 * x
# duration_modis_msavi = lambda x: -2.58 + 186.534 * x

volume_avhrr_ndvi = lambda x: -1.073 + 140.406 * x
duration_avhrr_ndvi = lambda x: 0.626 + 83.35 * x

df_mag_modis_ndwi['volume (mM^3)'] =volume_modis_ndwi(df_mag_modis_ndwi['Magnitude'])
df_mag_modis_ndwi['duration (days)'] = duration_modis_ndwi(df_mag_modis_ndwi['Magnitude'])

# df_mag_modis_msavi['volume (mM^3)'] = volume_modis_msavi(df_mag_modis_msavi['Magnitude'])
# df_mag_modis_msavi['duration (days)'] = duration_modis_msavi(df_mag_modis_msavi['Magnitude'])

df_mag_avhrr_ndvi['volume (mM^3)'] = volume_avhrr_ndvi(df_mag_avhrr_ndvi['Magnitude'])
df_mag_avhrr_ndvi['duration (days)']= duration_avhrr_ndvi(df_mag_avhrr_ndvi['Magnitude'])

df_mag_modis_ndwi= df_mag_modis_ndwi.drop(["date", "Magnitude","month"], axis=1)
df_mag_avhrr_ndvi= df_mag_avhrr_ndvi.drop(["date", "Magnitude","month"], axis=1)

satel = 'mod'
vi = 'MSAVI'

df1 = msavi_modis

# Get the start and end dates from the DataFrame
start = msavi_modis['date'].min().strftime('%Y-%m-%d')
end = msavi_modis['date'].max().strftime('%Y-%m-%d')

# Calculate the number of years and other variables
Y = (msavi_modis['date'].max().year - msavi_modis['date'].min().year) + 1
N = Y * 73
z = int(len(msavi_modis) / 10)
yt1, m1, d1 = msavi_modis['date'].min().strftime('%Y-%m-%d').split('-')
yt2, m2, d2 = msavi_modis['date'].max().strftime('%Y-%m-%d').split('-')

# Convert the year, month, and day strings to integers
yt1, m1, d1, yt2, m2, d2 = int(yt1), int(m1), int(d1), int(yt2), int(m2), int(d2)




df = df1
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date', drop = True)
df = df.sort_index()
df = df[start:end]
df = df.fillna(0)

TimeSeries_LST2 =  list(df[vi])   
TimeSeries_DATE2 = list(df.index)

while len(TimeSeries_LST2) > z*10:
    l = len(TimeSeries_LST2)
    TimeSeries_LST2.pop(-1)
    TimeSeries_DATE2.pop(-1)
    
nd = TimeSeries_LST2
time= TimeSeries_DATE2
df = pd.DataFrame({'date' : time, vi : nd})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date') 
df = df.sort_index()


def smooth(TS):
    N_TS = len(TS)
    
    N = 73
    #prepare the a and b coeficiants for the TFA
    Nharmonics = 3
    a = [0., 0., 0.]
    b = [0., 0., 0.]

    for t in range(N_TS):
        for p in range(Nharmonics):
            a[p] = a[p] + (2*TS[t]*cos(2*pi*(p+1)*(t+1)/N))
            b[p] = b[p] + (2*TS[t]*sin(2*pi*(p+1)*(t+1)/N))
            
    a = [a[i]/N_TS for i in range(len(a))] 
    b = [b[i]/N_TS for i in range(len(b))] 

    #prepare the first 3 harmonics of the TFA
    H0 = sum(TS) / N_TS
    TFA = [0. for j in range(N_TS)]
    H = [[0. for k in range(N_TS)] for i in range(Nharmonics)] 
    omega = [0. for j in range(Nharmonics)]   
    Phase = [0. for j in range(Nharmonics)]    
    Amplitude = [0. for j in range(Nharmonics)]
    
    for p in range(Nharmonics):
        omega[p] = 2. * pi * (p+1) / N
        Phase[p] = atan2( -b[p], a[p] )
        Amplitude[p] = sqrt( a[p]**2 + b[p]**2 )
        for t in range(N_TS):
            H[p][t] = Amplitude[p] * cos(omega[p]*(t+1) + Phase[p])

    for t in range(N_TS):
        TFA[t] = H0 + H[0][t] + H[1][t] + H[2][t]
    
    H1 = H[0][0:73]   
    H2 = H[1][0:73]   
    H3 = H[2][0:73]   
    return TFA, H0, Amplitude, Phase, H1, H2, H3

    


#we the data from ee  to 10 list of 46 point
chunks=[TimeSeries_LST2[x:x+z] for x in range(0, len(TimeSeries_LST2), z)]
#we trun the list into a matrix arry in order to make avg vale for each day in a year
new = np.array(chunks)
neww = np.array(new)
#print(new)
#we put no data vale on 1726.83 in order to deal with no data in the time series data from ee
#1726.83 is no data beacsoa 9999 * *0.02-273.15
noData = 1726.83
maskedvale = np.ma.masked_equal(new,noData)

#we do mean on the matriex so we get in the end 46 point of data with no missing data
mean = maskedvale.mean(0)

#IN THIS PART WE MAKE THE interpolating TO 73 point in a time seirs  form 46 in the orginal data

N82 = range(1, z+1)#range of 46 point 
y1 = mean
x = np.array(N82)
y = np.array(y1)

new_length = 73 #73 we set the range of the point we get from the interpolate to 73 
new_x = np.linspace(x.min(), x.max(), new_length)
new_y = sp.interpolate.interp1d(x, y, kind='cubic')(new_x)

n73fourier_vale = smooth(new_y)
LST_TFA       = n73fourier_vale[0]
LST_mean      = n73fourier_vale[1]
LST_Amplitude = n73fourier_vale[2]
LST_Phase     = n73fourier_vale[3]
H1            = n73fourier_vale[4]
H2            = n73fourier_vale[5]
H3            = n73fourier_vale[6]

#we set the vale
#plot the vale after the fourier
#make range of a year

julian_date = range(0,365,5)
Ymean = np.full(73,LST_mean)

import statsmodels.api as sm
df['index'] = df.index
y = df[vi].tolist()
x = df["index"].tolist()
n=len(df)
x = list(range(0,n))
lowess = sm.nonparametric.lowess(y, x, frac=.1)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]


df[vi+'_lowess'] = lowess_y

df[vi+'_less_trend'] = df[vi] - df[vi+'_lowess']
y2 = df[vi+'_less_trend'].tolist()
x2 = df["index"].tolist()
n=len(df)
x2 = list(range(0,n))
lowess = sm.nonparametric.lowess(y2, x2, frac=.02)
lowess_x2 = list(zip(*lowess))[0]
lowess_y2 = list(zip(*lowess))[1]


df[vi+'_less_trend'] = lowess_y2


import datetime
# initializing dates
test_date1 = datetime.datetime(1982, 1, 1)
test_date2 = datetime.datetime(2022, 12, 22)
                


temp = []
  
# getting diff.
diff = ( test_date2 - test_date1) // N
for idx in range(0, N):
      
    # computing new dates
    temp.append((test_date1 + idx * diff))
  
# using strftime to convert to userfriendly 
# format
res = []
for sub in temp:
  #res.append(sub.strftime("%Y/%m/%d %H:%M:%S"))
  res.append(sub.strftime("%Y/%m/%d"))
  

date_list_for_N73 =[]
for date in res:
  
  date_list_for_N73.append(datetime.datetime.strptime(str(date[0:4])+str(date[5:7])+str(date[8:10]), '%Y%m%d'))

#print(date_list_for_N73)

import math
to_forier = list(df[vi+'_less_trend'])
for i in to_forier:
  if math.isnan(i) == True:
    to_forier[i] = 0.04409690393354137
  else:
    continue 


#we the data from ee  to 10 list of 46 point
chunks=[to_forier[x:x+z] for x in range(0, len(to_forier), z)]
#we trun the list into a matrix arry in order to make avg vale for each day in a year
new = np.array(chunks)
#we put no data vale on 1726.83 in order to deal with no data in the time series data from ee
#1726.83 is no data beacsoa 9999 * *0.02-273.15
noData = 1726.83
maskedvale = np.ma.masked_equal(new,noData)
#we do mean on the matriex so we get in the end 46 point of data with no missing data
mean = maskedvale.mean(0)

#IN THIS PART WE MAKE THE interpolating TO 73 point in a time seirs  form 46 in the orginal data

N46 = range(1, z+1)#range of 46 point 
y1 = mean
x = np.array(N46)
y = np.array(y1)

new_length = 73 #73 we set the range of the point we get from the interpolate to 73 
new_x = np.linspace(x.min(), x.max(), new_length)
new_y = sp.interpolate.interp1d(x, y, kind='cubic')(new_x)

n73fourier_vale = smooth(new_y)
LST_TFA       = n73fourier_vale[0]


LST_TFA_Y = LST_TFA*Y

# dictionary of lists 
dict = {'date': date_list_for_N73, vi: LST_TFA_Y, } 
    
df_f = pd.DataFrame(dict)
df_f['date'] = pd.to_datetime(df_f['date'])

df_f.set_index('date', inplace=True) 
df_f = df_f.resample('D').pad() 
df_f = df_f[:(z*10)]


pd.set_option('display.max_rows', None)
df['fourier'] = df_f[vi]
k=0
for i in df[vi]:
  df['fourier'][k] = df_f[vi][k]
  k+=1
df['fourier'] =df['fourier'].interpolate()
list_ye = range(2001,2022)

df['area'] = df[vi+'_less_trend'] - df['fourier']

k=0
for i in df['area']:
  if i<=0 :
    df['area'][k]=0
    k+=1  
  elif i>0:
    k+=1
  
from datetime import datetime
from scipy.integrate import simps
import numpy as np
dict_integrate = {}
for year in range(yt1-1,yt2):
  slice = df['area'][(datetime(year,hydrological_year,1)):(datetime(year+1,hydrological_year,1))]  #for different hydrology year
  y = np.array(slice)
  if y.any() == False:  # check if y is empty
    I = 0
  else:
    I = simps(y,dx=12)
  if hydrological_year < 6:
    dict_integrate[(year)] = I  #for different hydrology year
  else:
    dict_integrate[(year+1)] = I #for different hydrology year


x = list(dict_integrate.keys())

y = list(dict_integrate.values())


df_integrate = pd.DataFrame()
df_integrate['year'] = list(dict_integrate.keys())
df_integrate['area'] = list(dict_integrate.values())
dev = df_integrate['area'].std()
df_integrate['std'] = 0.56

k=0
for i in df_integrate['area']:
  df_integrate['std'][k] = (i/dev)
  k+=1

df_gsi_modis_msavi = df_integrate
df_gsi_modis_msavi = df_gsi_modis_msavi[df_gsi_modis_msavi['std'] > 1]

#equations of GSI values:
volume_modis_msavi = lambda x: -4.55 + 0.369 * x
duration_modis_msavi = lambda x: 2.586 + 0.095 * x

df_gsi_modis_msavi['volume (mM^3)'] =volume_modis_msavi(df_gsi_modis_msavi['area'])
df_gsi_modis_msavi['duration (days)'] = duration_modis_msavi(df_gsi_modis_msavi['area'])
df_gsi_modis_msavi= df_gsi_modis_msavi.drop(["std", "area"], axis=1)

vi = 'MSAVI'

df1 = msavi_landsat

# Get the start and end dates from the DataFrame
start = msavi_landsat['date'].min().strftime('%Y-%m-%d')
end = msavi_landsat['date'].max().strftime('%Y-%m-%d')

# Calculate the number of years and other variables
Y = (msavi_landsat['date'].max().year - msavi_landsat['date'].min().year) + 1
N = Y * 73
z = int(len(msavi_landsat) / 10)
yt1, m1, d1 = msavi_landsat['date'].min().strftime('%Y-%m-%d').split('-')
yt2, m2, d2 = msavi_landsat['date'].max().strftime('%Y-%m-%d').split('-')

# Convert the year, month, and day strings to integers
yt1, m1, d1, yt2, m2, d2 = int(yt1), int(m1), int(d1), int(yt2), int(m2), int(d2)



df = df1
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date', drop = True)
df = df.sort_index()
df = df[start:end]
df = df.fillna(0)

TimeSeries_LST2 =  list(df[vi])   
TimeSeries_DATE2 = list(df.index)

while len(TimeSeries_LST2) > z*10:
    l = len(TimeSeries_LST2)
    TimeSeries_LST2.pop(-1)
    TimeSeries_DATE2.pop(-1)
    
nd = TimeSeries_LST2
time= TimeSeries_DATE2
df = pd.DataFrame({'date' : time, vi : nd})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date') 
df = df.sort_index()


def smooth(TS):
    N_TS = len(TS)
    
    N = 73
    #prepare the a and b coeficiants for the TFA
    Nharmonics = 3
    a = [0., 0., 0.]
    b = [0., 0., 0.]

    for t in range(N_TS):
        for p in range(Nharmonics):
            a[p] = a[p] + (2*TS[t]*cos(2*pi*(p+1)*(t+1)/N))
            b[p] = b[p] + (2*TS[t]*sin(2*pi*(p+1)*(t+1)/N))
            
    a = [a[i]/N_TS for i in range(len(a))] 
    b = [b[i]/N_TS for i in range(len(b))] 

    #prepare the first 3 harmonics of the TFA
    H0 = sum(TS) / N_TS
    TFA = [0. for j in range(N_TS)]
    H = [[0. for k in range(N_TS)] for i in range(Nharmonics)] 
    omega = [0. for j in range(Nharmonics)]   
    Phase = [0. for j in range(Nharmonics)]    
    Amplitude = [0. for j in range(Nharmonics)]
    
    for p in range(Nharmonics):
        omega[p] = 2. * pi * (p+1) / N
        Phase[p] = atan2( -b[p], a[p] )
        Amplitude[p] = sqrt( a[p]**2 + b[p]**2 )
        for t in range(N_TS):
            H[p][t] = Amplitude[p] * cos(omega[p]*(t+1) + Phase[p])

    for t in range(N_TS):
        TFA[t] = H0 + H[0][t] + H[1][t] + H[2][t]
    
    H1 = H[0][0:73]   
    H2 = H[1][0:73]   
    H3 = H[2][0:73]   
    return TFA, H0, Amplitude, Phase, H1, H2, H3

    


#we the data from ee  to 10 list of 46 point
chunks=[TimeSeries_LST2[x:x+z] for x in range(0, len(TimeSeries_LST2), z)]
#we trun the list into a matrix arry in order to make avg vale for each day in a year
new = np.array(chunks)
neww = np.array(new)
#print(new)
#we put no data vale on 1726.83 in order to deal with no data in the time series data from ee
#1726.83 is no data beacsoa 9999 * *0.02-273.15
noData = 1726.83
maskedvale = np.ma.masked_equal(new,noData)

#we do mean on the matriex so we get in the end 46 point of data with no missing data
mean = maskedvale.mean(0)

#IN THIS PART WE MAKE THE interpolating TO 73 point in a time seirs  form 46 in the orginal data

N82 = range(1, z+1)#range of 46 point 
y1 = mean
x = np.array(N82)
y = np.array(y1)

new_length = 73 #73 we set the range of the point we get from the interpolate to 73 
new_x = np.linspace(x.min(), x.max(), new_length)
new_y = sp.interpolate.interp1d(x, y, kind='cubic')(new_x)

n73fourier_vale = smooth(new_y)
LST_TFA       = n73fourier_vale[0]
LST_mean      = n73fourier_vale[1]
LST_Amplitude = n73fourier_vale[2]
LST_Phase     = n73fourier_vale[3]
H1            = n73fourier_vale[4]
H2            = n73fourier_vale[5]
H3            = n73fourier_vale[6]

#we set the vale
#plot the vale after the fourier
#make range of a year

julian_date = range(0,365,5)
Ymean = np.full(73,LST_mean)

import statsmodels.api as sm
df['index'] = df.index
y = df[vi].tolist()
x = df["index"].tolist()
n=len(df)
x = list(range(0,n))
lowess = sm.nonparametric.lowess(y, x, frac=.1)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]


df[vi+'_lowess'] = lowess_y

df[vi+'_less_trend'] = df[vi] - df[vi+'_lowess']
y2 = df[vi+'_less_trend'].tolist()
x2 = df["index"].tolist()
n=len(df)
x2 = list(range(0,n))
lowess = sm.nonparametric.lowess(y2, x2, frac=.02)
lowess_x2 = list(zip(*lowess))[0]
lowess_y2 = list(zip(*lowess))[1]


df[vi+'_less_trend'] = lowess_y2


import datetime
# initializing dates
test_date1 = datetime.datetime(1982, 1, 1)
test_date2 = datetime.datetime(2022, 12, 22)
                


temp = []
  
# getting diff.
diff = ( test_date2 - test_date1) // N
for idx in range(0, N):
      
    # computing new dates
    temp.append((test_date1 + idx * diff))
  
# using strftime to convert to userfriendly 
# format
res = []
for sub in temp:
  #res.append(sub.strftime("%Y/%m/%d %H:%M:%S"))
  res.append(sub.strftime("%Y/%m/%d"))
  

date_list_for_N73 =[]
for date in res:
  
  date_list_for_N73.append(datetime.datetime.strptime(str(date[0:4])+str(date[5:7])+str(date[8:10]), '%Y%m%d'))

#print(date_list_for_N73)

import math
to_forier = list(df[vi+'_less_trend'])
for i in to_forier:
  if math.isnan(i) == True:
    to_forier[i] = 0.04409690393354137
  else:
    continue 


#we the data from ee  to 10 list of 46 point
chunks=[to_forier[x:x+z] for x in range(0, len(to_forier), z)]
#we trun the list into a matrix arry in order to make avg vale for each day in a year
new = np.array(chunks)
#we put no data vale on 1726.83 in order to deal with no data in the time series data from ee
#1726.83 is no data beacsoa 9999 * *0.02-273.15
noData = 1726.83
maskedvale = np.ma.masked_equal(new,noData)
#we do mean on the matriex so we get in the end 46 point of data with no missing data
mean = maskedvale.mean(0)

#IN THIS PART WE MAKE THE interpolating TO 73 point in a time seirs  form 46 in the orginal data

N46 = range(1, z+1)#range of 46 point 
y1 = mean
x = np.array(N46)
y = np.array(y1)

new_length = 73 #73 we set the range of the point we get from the interpolate to 73 
new_x = np.linspace(x.min(), x.max(), new_length)
new_y = sp.interpolate.interp1d(x, y, kind='cubic')(new_x)

n73fourier_vale = smooth(new_y)
LST_TFA       = n73fourier_vale[0]


LST_TFA_Y = LST_TFA*Y

# dictionary of lists 
dict = {'date': date_list_for_N73, vi: LST_TFA_Y, } 
    
df_f = pd.DataFrame(dict)
df_f['date'] = pd.to_datetime(df_f['date'])

df_f.set_index('date', inplace=True) 
df_f = df_f.resample('D').pad() 
df_f = df_f[:(z*10)]


pd.set_option('display.max_rows', None)
df['fourier'] = df_f[vi]
k=0
for i in df[vi]:
  df['fourier'][k] = df_f[vi][k]
  k+=1
df['fourier'] =df['fourier'].interpolate()
list_ye = range(2001,2022)

df['area'] = df[vi+'_less_trend'] - df['fourier']

k=0
for i in df['area']:
  if i<=0 :
    df['area'][k]=0
    k+=1  
  elif i>0:
    k+=1
  
from datetime import datetime
from scipy.integrate import simps
import numpy as np

dict_integrate = {}
for year in range(yt1-1,yt2):
  slice = df['area'][(datetime(year,hydrological_year,1)):(datetime(year+1,hydrological_year,1))]  #for different hydrology year
  y = np.array(slice)
  if y.any() == False:  # check if y is empty
    I = 0
  else:
    I = simps(y,dx=12)
  if hydrological_year < 6:
    dict_integrate[(year)] = I  #for different hydrology year
  else:
    dict_integrate[(year+1)] = I #for different hydrology year

x = list(dict_integrate.keys())

y = list(dict_integrate.values())


df_integrate = pd.DataFrame()
df_integrate['year'] = list(dict_integrate.keys())
df_integrate['area'] = list(dict_integrate.values())
dev = df_integrate['area'].std()
df_integrate['std'] = 0.56

k=0
for i in df_integrate['area']:
  df_integrate['std'][k] = (i/dev)
  k+=1

df_gsi_landsat_msavi = df_integrate
df_gsi_landsat_msavi = df_gsi_landsat_msavi[df_gsi_landsat_msavi['std'] > 1]

nan_df = pd.DataFrame({'volume (mM^3)': [float('nan')]*len(df_gsi_landsat_msavi),
                       'duration (days)': [float('nan')]*len(df_gsi_landsat_msavi)})

# use assign method to add new columns to df_gsi_landsat_msavi
df_gsi_landsat_msavi = df_gsi_landsat_msavi.assign(**nan_df)
df_gsi_landsat_msavi= df_gsi_landsat_msavi.drop(["std", "area"], axis=1)

#merge the dfs of pre modis era
merged_df = pd.merge(df_gsi_landsat_msavi, df_mag_avhrr_ndvi, on="year", how="outer", suffixes=("_df1", "_df2"))
merged_df = merged_df.drop_duplicates(subset="year", keep="last")
merged_df = merged_df.drop(merged_df.filter(regex='_df1').columns, axis=1)
merged_df = merged_df.rename(columns=lambda x: x.replace("_df2", ""))
merged_df = merged_df.set_index('year').fillna(df_gsi_landsat_msavi.set_index('year')).reset_index()#merged_df =merged_df.sort_values(by ="year")
merged_df = merged_df.sort_values(by = 'year')
merged_df_pre = merged_df


#merge the dfs of modis era
merged_df = pd.merge(df_gsi_modis_msavi, df_mag_modis_ndwi, on="year", how="outer", suffixes=("_df1", "_df2"))
merged_df = merged_df.drop_duplicates(subset="year", keep="last")
merged_df = merged_df.drop(merged_df.filter(regex='_df1').columns, axis=1)
merged_df = merged_df.rename(columns=lambda x: x.replace("_df2", ""))
merged_df = merged_df.set_index('year').fillna(df_gsi_modis_msavi.set_index('year')).reset_index()#merged_df =merged_df.sort_values(by ="year")
merged_df = merged_df.sort_values(by = 'year')
merged_df_modis = merged_df

#merge them both to get final resulsts of SatVits - Floods model:
floods_df = pd.concat([merged_df_pre,merged_df_modis ])
floods_df = floods_df[(floods_df['volume (mM^3)'] >= 0) | (floods_df['volume (mM^3)'].isna())]

# save the DataFrame as a CSV file
floods_df.to_csv('floods_'+river_name+'.csv', index=False)

# download the CSV file
files.download('floods_'+river_name+'.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>